## <div align="center"> <h2 align="center"> РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ </h1> </div>
## <div align="center"> <h3 align="center"> Факультет физико-математических и естественных наук </h2> </div>
## <div align="center"> <h3 align="center"> Кафедра математического моделирования и искусственного интеллекта </h2> </div>
## <div align="center"> <h3 align="center"> Дисциплина: МОЗИиИБ </h2> </div>
## Отчет по лабораторной работе № 8
## Студент: Евдокимов Максим Михайлович
## Группа: НФИмд-01-24
## Целочисленная арифметика многократной точности

# Задание

1. сложение неотрицательных целых чисел.
2. вычитание неотрицательных целых чисел.
3. умножение неотрицательных целых чисел.
4. быстрый столбик.
5. деление многоразрядных целых чисел.

In [1]:
# сложение неотрицательных целых чисел
function add_big_integers(a::Vector{Int}, b::Vector{Int})
    lenA, lenB = length(a), length(b)
    lenC, k = max(lenA, lenB), 0
    result = zeros(Int, lenC)
    
    # Дополняем меньшее число нулями слева
    if lenA < lenC
        a = vcat(zeros(Int, lenC - lenA), a)
    elseif lenB < lenC
        b = vcat(zeros(Int, lenC - lenB), b)
    end
    
    for i in lenC:-1:1
        sum = a[i] + b[i] + k
        result[i] = sum % 10
        k = sum ÷ 10
    end
    if k > 0
        return [k; result]
    else
        return result 
    end
end

add_big_integers (generic function with 1 method)

In [151]:
# вычитание неотрицательных целых чисел
function subtract_big_integers(a::Vector{Int}, b::Vector{Int})
    lenA, lenB = length(a), length(b)
    lenC, k = max(lenA, lenB), 0
    result = zeros(Int, lenC)
    if lenA < lenB || (lenA == lenB && a < b)
        a, b = b, a
        negative_result = true
    else
        negative_result = false
    end
    
    # Дополняем меньшее число нулями слева
    if lenA < lenC
        a = vcat(zeros(Int, lenC - lenA), a)
    elseif lenB < lenC
        b = vcat(zeros(Int, lenC - lenB), b)
    end
    
    for i in length(a):-1:1
        a_digit = a[i]
        b_digit = b[i]
        diff = a_digit - b_digit - k
        if diff < 0
            diff += 10
            k = 1
        else
            k = 0
        end
        result[i] = diff
    end
    
    while result[1] == 0 && length(result) > 1
        popfirst!(result)
    end
    return result
end

subtract_big_integers (generic function with 1 method)

In [3]:
# умножение неотрицательных целых чисел
function multiply_big_integers(a::Vector{Int}, b::Vector{Int})
    lenA, lenB = length(a), length(b)
    result = zeros(Int, lenA+lenB+1)
    lenC = max(lenA, lenB)
    
    for i in lenA:-1:1
        for j in lenB:-1:1
            product = a[i] * b[j]
            sum = result[i+j+1] + product
            result[i+j+1] = sum % 10
            result[i+j] += sum ÷ 10
        end
    end
    
    # Убираем ведущие нули
    while length(result) > 1 && result[1] == 0
        result = result[2:end]
    end
    return result
end

multiply_big_integers (generic function with 1 method)

In [162]:
# быстрый столбик для неотрицательных целых чисел
function karatsuba_multiply(a::Vector{Int}, b::Vector{Int})
    if length(a) <= 1 || length(b) <= 1
        return [a[1] * b[1]]
    end
    lenA, lenB, lenC = length(a), length(b), length(a)+length(b)
    m = max(lenA, lenB) ÷ 2
    high1, low1 = split_vector(a, m, lenA)
    high2, low2 = split_vector(b, m, lenB)
    
    z0 = multiply_big_integers(low1, low2) # a_0 * b_0
    z1 = multiply_big_integers(low1, high2) # a_0 * b_1
    z2 = multiply_big_integers(high1, low2) # a_1 * b_0
    z3 = multiply_big_integers(high1, high2) # a_1 * b_1
    
    el0, el1, el2 = vcat(z3, zeros(Int, m*2)), vcat(zeros(Int, m), add_big_integers(z1, z2), zeros(Int, m)), vcat(zeros(Int, m*2), z0)
    z = add_big_integers(add_big_integers(el0, el2), el1)
    if z[1] == 0
        return z[2:end]
    end
    return z 
end

# Функция для разделения вектора на две части
function split_vector(v::Vector{Int}, m::Int, l::Int)
    high = v[1:l-m]
    low = v[l-m+1:end]
    return high, low
end

split_vector (generic function with 2 methods)

In [161]:
# деление многоразрядных целых чисел
function divide_big_integers(a::Vector{Int}, b::Vector{Int})
    function vector_to_bigint(v::Vector{Int})
        result = BigInt(0)
        for digit in v
            result = result * 10 + digit
        end
        return result
    end
    
    a_big = vector_to_bigint(a)
    b_big = vector_to_bigint(b)

    result = a_big ÷ b_big
    remainder = a_big % b_big
    result_digits = reverse(digits(result))
    
    fractional_part = Vector{Int}()
    for _ in 1:10
        remainder *= 10
        digit = remainder ÷ b_big
        remainder %= b_big
        push!(fractional_part, digit)
    end
    
    return result_digits, fractional_part
end

divide_big_integers (generic function with 1 method)

In [159]:
# Наши значния
numb1, numb2 = rand(1:9, 12), rand(1:9, 12)

([4, 1, 3, 8, 1, 2, 7, 1, 4, 9, 5, 1], [3, 7, 1, 7, 6, 5, 6, 8, 2, 2, 7, 8])

In [164]:
sum = add_big_integers(numb1, numb2)
println("Сумма: ", sum)

diff = subtract_big_integers(numb1, numb2)
println("Разность: ", diff)

product = multiply_big_integers(numb1, numb2)
println("Произведение: ", product)

karatsuba_product = karatsuba_multiply(numb1, numb2)
println("Произведение (быстрый столбик): ", karatsuba_product)

quotient, remainder = divide_big_integers(numb1, numb2)
println("Целая часть: ", quotient)
println("Остаток: ", remainder)

Сумма: [7, 8, 5, 5, 7, 8, 3, 9, 7, 2, 2, 9]
Разность: [4, 2, 0, 4, 7, 0, 3, 2, 6, 7, 3]
Произведение: [1, 5, 3, 8, 4, 1, 3, 6, 6, 3, 0, 9, 0, 7, 0, 0, 4, 6, 3, 3, 8, 3, 7, 8]
Произведение (быстрый столбик): [1, 5, 3, 8, 4, 1, 3, 6, 6, 3, 0, 9, 0, 7, 0, 0, 4, 6, 3, 3, 8, 3, 7, 8]
Целая часть: [1]
Остаток: [1, 1, 3, 1, 0, 0, 8, 9, 8, 4]
